In [1]:
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()
diabetes.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
773,6,103,72,32,190,37.7,0.324,55,0,Female
774,1,71,48,18,76,20.4,0.323,22,0,Female
775,0,117,0,0,0,33.8,0.932,44,0,Female
776,4,154,72,29,126,31.3,0.338,37,0,Female
777,5,147,78,0,0,33.7,0.218,65,0,Female


In [3]:
dupl_columns = list(diabetes.columns)
mask = diabetes.duplicated(subset = dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')
diabetes_dedupped = diabetes.drop_duplicates()
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')
diabetes_dedupped.tail()

Число найденных дубликатов: 10
Результирующее число записей: 768


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
763,5,139,64,35,140,28.6,0.411,26,0,Female
764,1,96,122,0,0,22.4,0.207,27,0,Female
765,10,101,86,37,0,45.6,1.136,38,1,Female
766,0,141,0,0,0,42.4,0.205,29,1,Female
767,0,125,96,0,0,22.5,0.262,21,0,Female


In [4]:
low_imformation_col = []
for col in diabetes_dedupped.columns:
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes_dedupped[col].nunique()/diabetes_dedupped[col].count()
    if top_freq > 0.95:
        low_imformation_col.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_imformation_col.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
new_diabetes_dedupped = diabetes_dedupped.drop(low_imformation_col, axis = 1)
print(f'Результирующее число признаков: {new_diabetes_dedupped.shape[1]}')
    

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


In [5]:
new_diabetes_dedupped['Glucose'] = new_diabetes_dedupped['Glucose'].apply(lambda x: np.nan if x == 0 else x)
new_diabetes_dedupped['BloodPressure'] = new_diabetes_dedupped['BloodPressure'].apply(lambda x: np.nan if x == 0 else x)
new_diabetes_dedupped['SkinThickness'] = new_diabetes_dedupped['SkinThickness'].apply(lambda x: np.nan if x == 0 else x)
new_diabetes_dedupped['BMI'] = new_diabetes_dedupped['BMI'].apply(lambda x: np.nan if x == 0 else x)
new_diabetes_dedupped['Insulin'] = new_diabetes_dedupped['Insulin'].apply(lambda x: np.nan if x == 0 else x)
new_diabetes_dedupped.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [6]:
drop_data = new_diabetes_dedupped.copy()
thresh = drop_data.shape[0]*0.7
drop_data = drop_data.dropna(thresh = thresh, axis = 1)
print(drop_data.shape)

(768, 8)


In [7]:
m = drop_data.shape[1]
drop_data = drop_data.dropna(thresh = m-2, axis = 0)
print(drop_data.shape)

(761, 8)


In [8]:
print(drop_data.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  \
0            6     98.0           58.0           33.0  34.0   
1            2    112.0           75.0           32.0  35.7   
2            2    108.0           64.0            NaN  30.8   
3            8    107.0           80.0            NaN  24.6   
4            7    136.0           90.0            NaN  29.9   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.430   43        0  
1                     0.148   21        0  
2                     0.158   21        0  
3                     0.856   34        0  
4                     0.210   50        0  


In [9]:
values = {'SkinThickness':drop_data['SkinThickness'].median(),
          'BloodPressure':drop_data['BloodPressure'].median(),
          'Glucose':drop_data['Glucose'].median(),
          'BMI':drop_data['BMI'].median()}
drop_data = drop_data.fillna(values)
print (drop_data['SkinThickness'].mean().round(1))

29.1


In [10]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [11]:
outliers, cleaned = outliers_iqr(drop_data, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [12]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


In [13]:
outliers, cleaned = outliers_z_score(drop_data, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [14]:
outliers, cleaned = outliers_iqr(drop_data, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732


In [15]:
def outliers_iqr_mode(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [16]:
outliers, cleaned = outliers_iqr_mode(drop_data, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 0
Результирующее число записей: 761
